In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
input = input("Enter your value: ") 

Enter your value: Stranger things


In [3]:
response = requests.get('http://www.imdb.com/find?ref_=nv_sr_fn&q=' +input+ '! &s=tt')
#+ input+ helps in finding data of a movie dynamically entered by user

In [4]:
#response = requests.get('http://www.imdb.com/find?ref_=nv_sr_fn&q= Stranger Things ! &s=tt')


In [5]:
soup = BeautifulSoup(response.content, 'lxml')
# Fetching the movie id so that the browser can be directed to that webpage. Note out of many searches we are taking the 1st search result
table = soup.find('table',class_='findList')
movieid = table.tr.a['href']
movieid

'/title/tt4574334/'

In [6]:
movielink = "http://www.imdb.com" + movieid

moviepage = requests.get(movielink)

soup2 = BeautifulSoup(moviepage.content, 'lxml')

In [8]:
# Title
uncleaned = soup2.find('div', class_ = 'title_wrapper').h1.text
#it contains some weird text and extra &nbsp at end
uncleaned =uncleaned.replace('\xa0','')
title = uncleaned.strip()
title

'Stranger Things'

In [168]:
#imdb rating
imdbcounter=1
try:
    imdb_rating = float(soup2.select('.ratingValue span')[0].text)
except:
    imdbcounter=0
if(imdbcounter !=0):
    print(imdb_rating)
else:
    print("imdb rating is not given on imdb page")

7.6


In [185]:
#metascore
try:
    metascore = float(soup2.select('.metacriticScore span')[0].text) if float(soup2.select('.metacriticScore span')[0].text) else None
except Exception as e:
    metascore=None
metascore

40.0

In [170]:
# Extracting cast values
namelist=[]
oddnames=[]
evennames =[]
ttag = soup2.find_all('tr', class_= 'odd')
for i in ttag:
    namelist.append( i.find('td', attrs={'class': None}))

for i in namelist:
    if (i.a) is None:
        pass
    else:
        oddnames.append(i.a.string.strip())
        
ttag = soup2.find_all('tr', class_= 'even')
namelist=[]

for i in ttag:
    namelist.append( i.find('td', attrs={'class': None}))

for i in namelist:
    if (i.a) is None:
        pass
    else:
        evennames.append(i.a.string.strip())

cast = oddnames+evennames
cast

['Sushant Singh Rajput',
 'Amit Sadh',
 'Digvijay Deshmukh',
 'Manav Kaul',
 'Muni Jha',
 'Amitabh Srivastava',
 'Irfan Khan',
 'Rajkummar Rao',
 'Amrita Puri',
 'Asif Basra',
 'Ashish Kakkad',
 'Morli Patel',
 'Bina Shah']

In [171]:
# Directors
directors=[]
director_container = soup2.find('div',class_ = 'credit_summary_item')

if((director_container.find('h4').string =='Director:') or (director_container.find('h4').string =='Creator:') or
(director_container.find('h4').string =='Creators:') or (director_container.find('h4').string =='Directors:')) :
    directors_container = director_container.find_all('a')
    for i in directors_container:
        directors.append(i.string)
else:
    pass
    
directors
    



['Abhishek Kapoor']

## To get value of attrs use .attrs

In [172]:
#country
country=[]
titledetails = soup2.find('div',id='titleDetails')
titleheading = titledetails.find_all('a')
for i in range(0,len(titleheading)):
    if('country_of_origin' in titleheading[i]['href']):
        country.append(titleheading[i].string)
country

['India']

In [173]:
#language
languages=[]
titledetails2 = soup2.find('div',id='titleDetails')
titleheading2 = titledetails2.find_all('a')
for i in range(0,len(titleheading2)):
    if('primary_language' in titleheading2[i]['href']):
        languages.append(titleheading2[i].string)
languages

['Hindi']

## Below is a way to find according to an attribute in tag-> consider that as dictionary

In [174]:
#Total number of votes
try:
    votes = soup2.find('span', {'itemprop':'ratingCount'}).string
    votes = votes.replace(',',"")
    votes = int(votes)
except Exception as e:
    votecounter=0
if(votecounter!=0):
    print(votes)
else:
    print("Number of votes are not given on imdb page")


27012


## next_sibling used to get text just after a tag closes

In [175]:
# release date
dateheading = soup2.select('.txt-block h4')
for i in range(0,len(dateheading)):
    if(dateheading[i].string=='Release Date:'):
        print(dateheading[i].next_sibling.strip())
        


22 February 2013 (India)


In [176]:
# genres
genres=[]
wrapper = soup2.select_one('.title_wrapper .subtext')
links = wrapper.find_all('a')
for i in links:
    if 'genres' in i['href']:
        genres.append(i.string)
        
genres

['Drama', 'Sport']

In [177]:
#duration-> This considers avg duration shown on imdb page for series
wrapper = soup2.select_one('.title_wrapper .subtext')
duration = wrapper.find('time').string.strip()
duration

'2h'

In [178]:
# Type, Duration/Seasons
durncounter=1
wrapper = soup2.select_one('.title_wrapper .subtext')
links = wrapper.find_all('a')
links
for i in range(0,len(links)):
    if 'releaseinfo' in links[i]['href']:
        value = links[i].string.strip()
if value.startswith('TV '):
    type = 'TV Series'
    seasons = soup2.select('.seasons-and-year-nav a')[0].string
    duration = seasons+" seasons"
else:
    type='Movie'
    try:
        duration = wrapper.find('time').string.strip()
    except Exception as e:
        durncounter=0
    

if(type == 'Movie') and durncounter!= 0:
    print("The duration is", duration)
elif(type == 'TV Series'):
    print("The number of seasons are ", duration)
else:
    print(" Duration is not given on imdb page")


The duration is 2h


In [180]:
# Storyline
# Here we have links inside story for some movies. So normal .strings wont work
story = soup2.select_one('#titleStoryLine div span').get_text().strip()
story
if(story!=''):
    print("\nHere is a description of the storyline - \n", story)
else:
    print("\nStory description is not given on imdb page")
    


Here is a description of the storyline - 
 The story goes around the three friends who start a business, also providing coaching classes and tuition. Omi asks for help with his uncle, who is also a political leader, to start this business. Govind provides tuition classes to some children and Ishaan gives cricket coaching. They come to meet a boy, Ali who is gifted with hyper-reflex. Ishaan then starts special coaching for the boy so that the country gets a new star player. But unexpected things happen suddenly and the story takes a turn.


In [181]:
# Certificate
wrapper = soup2.select_one('.title_wrapper .subtext')
wrapper.next_element.strip()
if(certificate != ''):
    print("\nThe {} has been given {} certificate".format(type,certificate))
else:
    print("Certificate details are not given on the imdb page")



The Movie has been given U certificate


# COMBINING ALL THE CODE

In [7]:
#title of movie/show
uncleaned = soup2.find('div', class_ = 'title_wrapper').h1.text
#it contains some weird text and extra &nbsp at end
uncleaned =uncleaned.replace('\xa0','')
title = uncleaned.strip()
print("Title is - {}".format(title))

# Type, Duration/Seasons
durncounter=1
wrapper = soup2.select_one('.title_wrapper .subtext')
links = wrapper.find_all('a')
links
for i in range(0,len(links)):
    if 'releaseinfo' in links[i]['href']:
        value = links[i].string.strip()
if value.startswith('TV '):
    type = 'TV Series'
    seasons = soup2.select('.seasons-and-year-nav a')[0].string
    duration = seasons+" seasons"
else:
    type='Movie'
    try:
        duration = wrapper.find('time').string.strip()
    except Exception as e:
        durncounter=0
    

if(type == 'Movie') and durncounter!= 0:
    print("\nThe duration is", duration)
elif(type == 'TV Series'):
    print("\nThe number of seasons are ", duration)
else:
    print("\nDuration is not given on imdb page")

#imdb rating
imdbcounter=1
try:
    imdb_rating = float(soup2.select('.ratingValue span')[0].text)
except Exception as e:
    imdbcounter=0
if(imdbcounter !=0):
    print("\nThe imdb rating of the {} is = {}".format(type,imdb_rating))
else:
    print("\nimdb rating is not given on imdb page")


#Total number of votes
votecounter=1
try:
    votes = soup2.find('span', {'itemprop':'ratingCount'}).string
    votes = votes.replace(',',"")
    votes = int(votes)
except Exception as e:
    votecounter=0
if(votecounter!=0):
    print("\nThe imdb rating is calculating on the basis of {} mumber of votes".format( votes))
else:
    print("\nNumber of votes are not given on imdb page")

#metascore    
try:
    metascore = float(soup2.select('.metacriticScore span')[0].text) if float(soup2.select('.metacriticScore span')[0].text) else None
    print("\nMetascore - ",metascore)
except Exception as e:
    metascore=None
    print("\nMetascore is not available in imdb site")

# cast values
namelist=[]
oddnames=[]
evennames =[]
ttag = soup2.find_all('tr', class_= 'odd')
for i in ttag:
    namelist.append( i.find('td', attrs={'class': None}))

for i in namelist:
    if (i.a) is None:
        pass
    else:
        oddnames.append(i.a.string.strip())
        
ttag = soup2.find_all('tr', class_= 'even')
namelist=[]

for i in ttag:
    namelist.append( i.find('td', attrs={'class': None}))

for i in namelist:
    if (i.a) is None:
        pass
    else:
        evennames.append(i.a.string.strip())

cast = oddnames+evennames
print("\nCast is as follows",cast)

# Extracting directors
directors=[]
director_container = soup2.find('div',class_ = 'credit_summary_item')

if((director_container.find('h4').string =='Director:') or (director_container.find('h4').string =='Creator:') or
(director_container.find('h4').string =='Creators:') or (director_container.find('h4').string =='Directors:')) :
    directors_container = director_container.find_all('a')
    for i in directors_container:
        directors.append(i.string)
    print("\nDirectors are", directors)
else:
    print("\nDirector information not given in imdb site")

# Country of production
country=[]
titledetails = soup2.find('div',id='titleDetails')
titleheading = titledetails.find_all('a')
for i in range(0,len(titleheading)):
    if('country_of_origin' in titleheading[i]['href']):
        country.append(titleheading[i].string)
print("\nProduction Countries-", country)
        
# Language
languages=[]
titledetails2 = soup2.find('div',id='titleDetails')
titleheading2 = titledetails2.find_all('a')
for i in range(0,len(titleheading2)):
    if('primary_language' in titleheading2[i]['href']):
        languages.append(titleheading2[i].string)
print("\n{} is available in Languages- {}".format(type, languages))

# release date
dateheading = soup2.select('.txt-block h4')
for i in range(0,len(dateheading)):
    if(dateheading[i].string=='Release Date:'):
        releasedate = dateheading[i].next_sibling.strip()
print("\n{} was released in- {}".format(type, releasedate))

        
# genres
genres=[]
wrapper = soup2.select_one('.title_wrapper .subtext')
links = wrapper.find_all('a')
for i in links:
    if 'genres' in i['href']:
        genres.append(i.string)
print("\nFollowing are the genres of the {} - {}".format(type,genres))

# Certificate
wrapper = soup2.select_one('.title_wrapper .subtext')
certificate = wrapper.next_element.strip()
if(certificate != ''):
    print("\nThe {} has been given {} certificate".format(type,certificate))
else:
    print("\nCertificate details are not given on the imdb page")
    
# Storyline
# Here we have links inside story for some movies. So normal .strings wont work
story = soup2.select_one('#titleStoryLine div span').get_text().strip()
if(story!=''):
    print("\nHere is a description of the storyline - \n", story)
else:
    print("\nStory description is not given on imdb page")
    

Title is - Stranger Things

The number of seasons are  4 seasons

The imdb rating of the TV Series is = 8.8

The imdb rating is calculating on the basis of 748198 mumber of votes

Metascore is not available in imdb site

Cast is as follows ['Winona Ryder', 'Finn Wolfhard', 'Gaten Matarazzo', 'Natalia Dyer', 'Joe Keery', 'Noah Schnapp', 'Priah Ferguson', 'Maya Hawke', 'Joe Chrest', 'David Harbour', 'Millie Bobby Brown', 'Caleb McLaughlin', 'Charlie Heaton', 'Cara Buono', 'Sadie Sink', 'Dacre Montgomery', 'Brett Gelman']

Directors are ['Matt Duffer', 'Ross Duffer']

Production Countries- ['USA']

TV Series is available in Languages- ['English', 'Russian']

TV Series was released in- 15 July 2016 (India)

Following are the genres of the TV Series - ['Drama', 'Fantasy', 'Horror']

The TV Series has been given 15 certificate

Here is a description of the storyline - 
 In a small town where everyone knows everyone, a peculiar incident starts a chain of events that leads to the disappearance